<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Aplicando-Pipeline" data-toc-modified-id="Aplicando-Pipeline-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aplicando Pipeline</a></span></li></ul></div>

In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.gridspec import GridSpec
try:
    import missingno as msno
except:
    !pip install missingno
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
import string
import sys
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
try:
    import urlextract
except:
    !pip install urlextract
from scipy.sparse import csr_matrix
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import time
import sys
import math
import itertools
import folium
from folium.plugins import Fullscreen
try:
    import geopandas as gpd
except:
    !pip install geopandas
import json
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

  Using cached https://files.pythonhosted.org/packages/83/c5/3cf9cdc39a6f2552922f79915f36b45a95b71fd343cfc51170a5b6ddb6e8/geopandas-0.7.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/56/57/462b634a5ab562201f4f208b3cd077b80e8e04359509ec294e22f7dd006c/pyproj-2.6.0-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/ea/55/61a5d274a210585b5d0c3dac81a82952a4baa7903e3642228d7a465fc340/Shapely-1.7.0-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/6d/42/f4a7cac53b28fa70e9a93d0e89a24d33e14826dad6644b699362ad84dde0/Fiona-1.8.13.post1.tar.gz
    Complete output from command python setup.py egg_info:
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\thipa\AppData\Local\Temp\pip-install-jcsvdq9_\fiona\


In [2]:
# Funções
def format_spines(ax, right_border=True):
    """
    Função responsável por formatar os eixos dos gráficos
    
    Input:
        ax: matplotlib axis
        right_border: flag booleano que indica a presença ou não da borda direita
    
    Output:
        None
    """    
    ax.spines['bottom'].set_color('#CCCCCC')
    ax.spines['left'].set_color('#CCCCCC')
    ax.spines['top'].set_visible(False)
    if right_border:
        ax.spines['right'].set_color('#CCCCCC')
    else:
        ax.spines['right'].set_color('#FFFFFF')
    ax.patch.set_facecolor('#FFFFFF')
    
def bar_plot(x, y, df, ax, colors='Blues_d', hue=False, value=False):
    """
    Função responsável por plotar, configurar e personalizar um gráfico de barras

    Input:
        x: feature a ser plotada no eixo x
        y: feature a ser plotada no eixo y
        df: dataframe utilizado
        colors: palette padrão "Blues_d"
        hue: feature de separação, padrão False
        value: flag que define se o rótulo das barras será valor (True) ou porcentagem (False)
        
    Returns:
        None
    """
    # Realizando contagem da feature numérica (em x ou em y)
    try:
        ncount = sum(df[y])
    except:
        ncount = sum(df[x])
    
    # Plotando gráfico no eixo
    if hue != False:
        ax = sns.barplot(x=x, y=y, data=df, palette=colors, hue=hue, ax=ax, ci=None)
    else:
        ax = sns.barplot(x=x, y=y, data=df, palette=colors, ax=ax, ci=None)

    # Configurando porcentagem ou valor
    for p in ax.patches:
        xp=p.get_bbox().get_points()[:,0]
        yp=p.get_bbox().get_points()[1,1]
        if value:
            ax.annotate('{:.2f}k'.format(yp/1000), (xp.mean(), yp), 
                    ha='center', va='bottom')
        else:
            ax.annotate('{:.1f}%'.format(100.*yp/ncount), (xp.mean(), yp), 
                    ha='center', va='bottom')
            
def add_series_working_days(series_name, df, date_col1, date_col2):
    """
    Função extremamente importante: calcula os dias úteis entre duas datas e retorna este valor
    como uma coluna adicional em um dataframe
    
    Input:
        series_names: nome da nova série criada para alocar a diferença em dias úteis
        df: conjunto de dados onde as colunas de data se encontram
        date_col1: coluna de datas 1
        date_col2: coluna de datas 2
        
    Returns:
        df_return: dataframe com coluna de dias úteis já inserida
    """
    # Calculando lista com diferença entre datas
    time_list = []
    idx = 0
    second_date_series = df[date_col2].values.astype('datetime64[D]')
    for date in df[date_col1].values.astype('datetime64[D]'):
        second_date = second_date_series[idx]
        try:
            workdays = np.busday_count(date, second_date)
        except:
            workdays = np.NaN
        time_list.append(workdays)
        idx += 1
    
    # Adicionando ao dataframe e eliminando valores nulos
    df_return = df.copy()
    df_return[series_name] = pd.Series(time_list)
    df_return.dropna(inplace=True)
    
    return df_return

def communicate_params(frete, entrega, estimativa):
    """
    Função adicional criada para auxiliar na plotagem gráfica do dashboard de análise por estado
    
    Input:
        frete: valor do frete a ser plotado no gráfico em formato de texto
        entrega: tempo médio de entrega a ser plotado no gráfico em formato de texto
        estimativa: diferença entre estimativa e entrega ser plotada no gráfico em formato de texto
    
    Returns:
        None
    """
    # 
    """axs[0, 0].text(0.5, 0.12, str(f'Frete Médio \nR${frete:.2f}'),
                   fontsize=13, ha='center')
    axs[0, 0].axis('off')"""
    
    # Comunicando frete médio
    ax1.text(0.23, 0.6, f'R${frete:.2f}', fontsize=65)
    ax1.text(0.22, 0.45, 'é o valor médio de frete', fontsize=12)
    ax1.text(0.24, 0.4, 'para compras online', fontsize=12)
    ax1.axis('off')
    
    # Comunicando tempo médio de entrega
    axs[0, 1].text(0.5, 0.12, 
                   str(f'Tempo Médio de Entrega \n{int(entrega)} dias úteis.'),
                   fontsize=13, ha='center')
    axs[0, 1].axis('off')
    
    # Comunicando diferença entre tempo de entrega e tempo estimado
    if mean_diff_estimative > 0:
        inf = 'antes'
    else:
        inf = 'depois'
    text_diff_estimative = f'Em média, entregas \nacontecem {int(estimativa)} dias úteis \
\n{inf} do período estimado.'
    axs[0, 2].text(0.5, 0.10, 
                   str(text_diff_estimative), fontsize=13, ha='center')
    axs[0, 2].axis('off')
    
def plot_param(df, col, title, xlim, n_row, n_col, y='customer_state', div_xlim=0, 
               one_axis=False, xlabel=[], ylabel='Estado'):
    """
    Função utilizada para plotar um estudo comparativo de acordo com o tema, 
    separando análises de dos 5 Melhores e 5 Piores em assuntos pré-determinados
    
    Input:
        df: dataframe com as informações
        col: coluna com a informação a ser analisada
        title: título da análise
        xlim: limite no eixo X dos números
        n_row: índice da linha onde a plotagem será realizada
        n_col: índice da coluna onde a plotagem será realizada
    """
    # Configurando eixos
    if one_axis:
        ax_top = axs[n_col]
        ax_last = axs[n_col+1]
    else:
        ax_top = axs[n_row, n_col]
        ax_last = axs[n_row+1, n_col]
    
    # Primeiro passo: plotando Top 5
    df.sort_values(by=col, ascending=False, inplace=True)
    top5 = df.iloc[:5, :]
    sns.barplot(x=col, y=y, data=top5, ci=None, palette='Blues_d', ax=ax_top)
    format_spines(ax_top, right_border=False)
    ax_top.set_title(title)
    ax_top.set_xlim(0, xlim)
    ax_top.set_xlabel(xlabel)
    if n_col > 0:
        ax_top.set_ylabel('')
    else:
        ax_top.set_ylabel(ylabel)
    
    # Segundo passo: plotando Top 5 inverso
    last5 = df.iloc[-5:, :]
    sns.barplot(x=col, y=y, data=last5, ci=None, palette='Blues_d', ax=ax_last)
    format_spines(ax_last, right_border=False)
    ax_last.set_title(title.replace('Maior', 'Menor'))
    if div_xlim > 0:
        ax_last.set_xlim(0, xlim/div_xlim)
    else:
        ax_last.set_xlim(0, xlim)
    ax_last.set_xlabel(xlabel)
    if n_col > 0:
        ax_last.set_ylabel('')
    else:
        ax_last.set_ylabel(ylabel)
        
def style_function(feature):
    """
    Função responsável por customizar mapas do folium
    """
    return {
        'fillColor': '#ffaf00',
        'color': 'grey',
        'weight': 1.5,
        'dashArray': '5, 5'
    }

def highlight_function(feature):
    """
    Função responsável por customizar mapas do folium
    """
    return {
        'fillColor': '#ffaf00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
        
def donut_plot(col, ax, df, labels, text='', flag_ruido = 0,
               colors=['navy', 'lightsteelblue', 'lightgreen', 'crimson', '']):
    """
    Função responsável por plotar um gráfico de rosca customizado
    
    Input:
        col: coluna a ser analisada e plotada no gráfico de rosca
        ax: matplotlib axis
        df: DataFrame contendo os dados
        labels: índice de labels da coluna a ser plotada
        text: texto a ser plotado no centro do gráfico de rosca
        flag_ruido: indica a quantidade de elementos a serem eliminados da plotagem
        colors: lista de cores para plotagem (4 cores default)
    """
    flag_ruido = flag_ruido * -1
    if flag_ruido < 0:
        sizes = df[col].value_counts().values[:flag_ruido]
        labels = labels[:flag_ruido]
    else:
        sizes = df[col].value_counts().values
    center_circle = plt.Circle((0,0), 0.80, color='white')
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.2f%%')
    ax.add_artist(center_circle)
    kwargs = dict(size=20, fontweight='bold', va='center')
    ax.text(0, 0, text, ha='center', **kwargs)
    
def text_process(c):
    """
    Função responsável por remover as pontuações e as stopwords das críticas.
    
    Input:
        c: crítica do cliente
    
    Output:
        lista sem pontuação e sem stopwords
    """
    # Retira pontuações
    nopunc = [char for char in c if char not in string.punctuation]

    # Junta-os para formar strings
    nopunc = ''.join(nopunc)
    
    # Remove as stopwords
    return [word.lower() for word in nopunc.split() if word.lower() not in stopwords.words('portuguese')]

def stem_processing(c):
    """
    Função responsável por realizarz o stemming nas críticas.
    
    Input:
        c: crítica do cliente
        
    Output:
        critica após aplicação do stemming
    """
    
    stemmer = RSLPStemmer()
    return list(map(lambda x: stemmer.stem(x), [word for word in c.split()]))

def create_dataset():
    """
    Função responsável por criar um dataset vazio para armazenamento das métricas
    
    Returns:
        DataFrame vazio
    """
    attributes = ['acc', 'prec', 'rec', 'f1', 'total_time']
    model_performance = pd.DataFrame({})
    for col in attributes:
        model_performance[col] = []
        
    return model_performance

def model_analysis(model, X, y, X_test, y_test, df_performance, cv=5, train=True):
    """
    Função responsável por aplicar métodos de avaliação de um modelo e salvar os resultados em um DataFrame
    
    Input:
        model: modelo a ser utilizado na avaliação
        X, y: Dados de treino, teste e labels
        df_performance: DataFrame vazio (gerado pela função create_dataset())
        cv: cross validation k folds

    Returns:
        Um objeto do tipo DataFrame com todas as métricas avaliadas
    """
    # Accuracy, precision, recall and f1_score on training set using cv
    t0_cv = time.time()
    acc = cross_val_score(model, X, y, cv=cv, scoring='accuracy').mean()
    prec = cross_val_score(model, X, y, cv=cv, scoring='precision').mean()
    rec = cross_val_score(model, X, y, cv=cv, scoring='recall').mean()
    f1 = cross_val_score(model, X, y, cv=cv, scoring='f1').mean()
    # Time spent on cross_validation prediction
    t1_cv = time.time()
    delta_time_cv = t1_cv-t0_cv
    
    # Evaluation using the test set
    t0_test = time.time()
    y_pred_test = model.predict(X_test)
    acc_test = accuracy_score(y_test, y_pred_test)
    prec_test = precision_score(y_test, y_pred_test)
    rec_test = recall_score(y_test, y_pred_test)
    f1_test = f1_score(y_test, y_pred_test)
    y_scores_test = model.predict_proba(X_test)[:, 1]
    # Time spent on test prediction
    t1_test = time.time()
    delta_time_test = t1_test-t0_test

    # Saving on dataframe
    performances = {}
    performances['acc'] = round(acc, 4)
    performances['prec'] = round(prec, 4)
    performances['rec'] = round(rec, 4)
    performances['f1'] = round(f1, 4)
    performances['total_time'] = round(delta_time_cv, 3)        
    df_performance = df_performance.append(performances, ignore_index=True)
    
    test_performances = {}
    test_performances['acc'] = round(acc_test, 4)
    test_performances['prec'] = round(prec_test, 4)
    test_performances['rec'] = round(rec_test, 4)
    test_performances['f1'] = round(f1_test, 4)
    test_performances['total_time'] = round(delta_time_test, 3)        
    df_performance = df_performance.append(test_performances, ignore_index=True)
    
    model_name = model.__class__.__name__
    df_performance.index = [model_name+' cv', model_name+' test']
    
    return df_performance

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    Esta função plota e personaliza uma matriz de confusão
    
    Input:
        cm: matriz de confusão gerada pela função do sklearn confusion_matrix(set, predictions)
        classes: target labels a serem plotados
        title: título do gráfico
        cmap: cores da matriz
    Output:
        Matriz de confusão personalizada
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=14)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    # Plot configuration
    thresh = cm.max() / 1.2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def plot_sentimento(model, frase):
    """
    Função responsável por receber uma frase pré-processada e um algoritmo de ML já treinado e 
    comunicar o resultado do sentimento em uma plotagem gráfica
    
    Input:
        model: modelo de classificação já treinado
        frase: frase teste já pré-processada
    
    Output:
        plotagem do resultado da predição utilizando matplotlib
    """
    # Predizendo
    pred = model.predict(frase)
    fig, ax = plt.subplots(figsize=(5, 3))
    if pred[0] == 1:
        text = 'Positivo'
        proba = 100 * round(model.predict_proba(frase)[0][1], 4)
        color = 'seagreen'
    else:
        text = 'Negativo'
        proba = 100 * round(model.predict_proba(frase)[0][0], 4)
        color = 'crimson'
    ax.text(0.5, 0.5, text, fontsize=50, ha='center', color=color)
    ax.text(0.5, 0.20, str(proba) + '%', fontsize=14, ha='center')
    ax.axis('off')
    ax.set_title('Sentimento', fontsize=14)
    plt.show()

In [3]:
df = pd.read_csv('olist_order_reviews_dataset.csv', encoding='UTF-8')
df = df.loc[:, ['review_comment_message']]
df.dropna(inplace=True)
df.head()

,review_comment_message
3,Recebi bem antes do prazo estipulado.
4,Parabéns lojas lannister adorei comprar pela I...
9,aparelho eficiente. no site a marca do aparelh...
12,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,"Vendedor confiável, produto ok e entrega antes..."


In [4]:
# Classe para implementação das Expressões Regulares
class ApplyRegex(BaseEstimator, TransformerMixin):
    
    def __init__(self, break_line=True, carriage_return=True, numbers=True, number_replacing='number', 
                 special_char=True, additional_spaces=True):
        self.break_line = break_line
        self.carriage_return = carriage_return
        self.numbers = numbers
        self.number_replacing = number_replacing
        self.special_char = special_char
        self.additional_spaces = additional_spaces
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = []
        for c in X:
            if self.break_line:
                c = re.sub('\n', ' ', c)
            if self.carriage_return:
                c = re.sub('\r', ' ', c)
            if self.numbers:
                c = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' '+self.number_replacing+' ', c)
            if self.special_char:
                c = re.sub(r'R\$', ' ', c)
                c = re.sub(r'\W', ' ', c)
            if self.additional_spaces:
                c = re.sub(r'\s+', ' ', c)
            X_transformed.append(c)
        return X_transformed

In [5]:
# Classe para remoção de Stopwords e pontuações
class StopWordsPunctuationRemoval(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = list(map(lambda c: text_process(c), X))
        X_transformed = list(map(lambda x: ' '.join(x), X_transformed))
        return X_transformed

In [6]:
# Classe para aplicação do Stemming
class TextStemming(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        """X_Series = pd.Series(X)
        X_transformed = X_Series.apply(stem_processing)
        X_transformed = X_transformed.apply(lambda x: ' '.join(x))
        return list(X_transformed.values)"""
        X_transformed = list(map(lambda c: stem_processing(c), X))
        X_transformed = list(map(lambda x: ' '.join(x), X_transformed))
        return X_transformed

In [7]:
# Classe para aplicação do TF-IDF
class TF_IDF(BaseEstimator, TransformerMixin):
    
    def __init__(self, max_features=300, stop_words=stopwords.words('portuguese')):
        self.max_features = max_features
        #self.min_df = min_df
        #self.max_df = max_df
        self.stop_words = stop_words
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        vectorizer = CountVectorizer(max_features=self.max_features, stop_words=self.stop_words)
        X_transformed = vectorizer.fit_transform(X)
        return X_transformed.toarray()

Após a definição de todas as classes responsáveis pela preparação dos dados, vamos consolidá-las em uma forma de **_Pipeline_**.

In [8]:
# Definindo Pipeline completo
preprocess_pipeline = Pipeline([
    ('regex_cleaner', ApplyRegex()),
    ('stopwords_punc_remover', StopWordsPunctuationRemoval())
    
    #('stemming', TextStemming()),
])

### Aplicando Pipeline

Definido um pipeline de pre-processamento, vamos coletar o dataset original, separar em conjuntos de _treino_ e _teste_ e, por fim, realizar todos os procedimentos

In [9]:
# Aplicando pipeline
X = df['review_comment_message']
X_preprocessed = preprocess_pipeline.fit_transform(X)

# Transformando textos preparados em DataFrame
df_comment = pd.DataFrame({
    'comentario': X_preprocessed
})
df_comment.head()

,comentario
0,recebi bem antes prazo estipulado
1,parabéns lojas lannister adorei comprar intern...
2,aparelho eficiente site marca aparelho impress...
3,pouco travando valor ta boa
4,vendedor confiável produto ok entrega antes prazo


In [10]:
# Gerando demais colunas do DataFrame (simulando situação real)
df_comment['ano_mes'] = [np.random.randint(201901, 201913) for i in df_comment.index]
df_comment['flag_status'] = [np.random.randint(1, 3) for i in df_comment.index]
df_comment['status'] = df_comment['flag_status'].apply(lambda x: 'APROVADO' if x == 1 else 'REPROVADO')
df_comment.drop('flag_status', axis=1, inplace=True)
df_comment['cluster'] = df_comment['status'].apply(lambda x: np.random.randint(0, 3) if x == 'APROVADO' \
                                                   else np.random.randint(0, 5))
df_comment = df_comment.loc[:, ['ano_mes', 'status', 'cluster', 'comentario']]
df_comment = df_comment.reset_index(level=0)
df_comment.head()

,index,ano_mes,status,cluster,comentario
0,0,201906,APROVADO,2,recebi bem antes prazo estipulado
1,1,201910,APROVADO,1,parabéns lojas lannister adorei comprar intern...
2,2,201905,APROVADO,2,aparelho eficiente site marca aparelho impress...
3,3,201907,APROVADO,1,pouco travando valor ta boa
4,4,201903,REPROVADO,2,vendedor confiável produto ok entrega antes prazo


In [11]:
# Criando outro DataFrame com comentário splitado e empilhado
df_stack = pd.DataFrame(df_comment['comentario'].str.split(' ', expand=True).stack())
df_stack.columns = ['stacked_text']
df_stack = df_stack.reset_index(level=0)
df_stack.head(10)

,level_0,stacked_text
0,0,recebi
1,0,bem
2,0,antes
3,0,prazo
4,0,estipulado
0,1,parabéns
1,1,lojas
2,1,lannister
3,1,adorei
4,1,comprar


In [12]:
# Unindo conjuntos
base = df_comment.merge(df_stack, how='left', left_on='index', right_on='level_0')
base.drop(['index', 'level_0', 'comentario'], axis=1, inplace=True)

# Transformando tipos primitivos
for col in base.columns:
    base[col] = base[col].astype(str)

# Adicionando flag para contagem
base['qtd'] = 1
base.head(20)

,ano_mes,status,cluster,stacked_text,qtd
0,201906,APROVADO,2,recebi,1
1,201906,APROVADO,2,bem,1
2,201906,APROVADO,2,antes,1
3,201906,APROVADO,2,prazo,1
4,201906,APROVADO,2,estipulado,1
5,201910,APROVADO,1,parabéns,1
6,201910,APROVADO,1,lojas,1
7,201910,APROVADO,1,lannister,1
8,201910,APROVADO,1,adorei,1
9,201910,APROVADO,1,comprar,1


In [13]:
# Comparando volumetrias
print(f'Volumetria base inicial: {len(df)}')
print(f'Volumetria base pivotada: {len(base)}')

print(f'\nCrescimento de {round((len(base) / len(df)), 1)} vezes')

Volumetria base inicial: 41753
Volumetria base pivotada: 307588

Crescimento de 7.4 vezes


In [14]:
# Volumetria por ano_mes (pensando em alguma redução)
base['ano_mes'].value_counts()

201906    26773
201905    26156
201911    26109
201907    26027
201909    26005
201908    25743
201902    25487
201912    25394
201904    25324
201910    24962
201901    24846
201903    24762
Name: ano_mes, dtype: int64

In [15]:
base.to_csv(r'C:\Users\thipa\Desktop\tbl_rsrc_stacked_words.txt', index=False)

In [16]:
nova_base = base.groupby(by=['stacked_text', 'ano_mes','status', 'cluster'],  
                         as_index=False).sum().sort_values(by=['ano_mes', 'status', 'cluster', 'qtd'],
                                                           ascending=[True, True, True, False])

nova_base.head()

,stacked_text,ano_mes,status,cluster,qtd
77124,produto,201901,APROVADO,0,256
64037,number,201901,APROVADO,0,168
74653,prazo,201901,APROVADO,0,122
35595,entrega,201901,APROVADO,0,90
5541,antes,201901,APROVADO,0,78


In [17]:
# Comparando volumetrias
print('-- NOVA ABORDAGEM -- \n')
print(f'Volumetria base inicial: {len(df)}')
print(f'Volumetria nova base: {len(nova_base)}')

print(f'\nCrescimento de {round((len(nova_base) / len(df)), 1)} vezes')

-- NOVA ABORDAGEM -- 

Volumetria base inicial: 41753
Volumetria noda base: 103367

Crescimento de 2.5 vezes


In [18]:
nova_base['ano_mes'].value_counts()

201905    8881
201906    8827
201909    8743
201908    8698
201911    8691
201907    8633
201902    8563
201910    8531
201912    8515
201904    8484
201903    8466
201901    8335
Name: ano_mes, dtype: int64

In [19]:
base.to_csv(r'C:\Users\thipa\Desktop\tbl_rsrc_stacked_words_nova.txt', index=False)